# Time Travel Debugger project

## Personal Information

In [1]:
PROJECT_TYPE = 2
NAME = ["Daniel Gusenburger", "Daniel Tabellion"]
ID = ["2544941", "2555742"]
IMPLEMENTED = set()

In [2]:
print(f"Implemented features: {IMPLEMENTED}")

Implemented features: set()


## Implementation

In [6]:
import Debugger

class TimeTravelDebugger(Debugger):
    
    def __init__(self, *args, **kwargs):
        pass
    
    def __enter__(self):
        pass
    
    def __exit__(self, *args, **kwargs):
        pass

ModuleNotFoundError: No module named 'Timer'

## Presentation

## Summary